In [1]:
import FluidSim
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation
from scipy import ndimage
import cv2

def smooth(data, kernel_size = 3):
    kernel = np.ones([kernel_size, kernel_size]) / (kernel_size*kernel_size)
    return ndimage.convolve(data, kernel)

In [9]:
sim = FluidSim.FluidSim(200,200,2,2, dt=0.0001)
sim.timestep()

dye = np.zeros_like(sim.u)

upscaleMult = 5
arrowSpacing = 5

imageSize = [sim.nx*upscaleMult, sim.ny * upscaleMult]

upscaledIndices = (sim.indices * upscaleMult)[:,::arrowSpacing,::arrowSpacing]

rowsUpsc = upscaledIndices[1,:,:].flatten()
colsUpsc = upscaledIndices[0,:,:].flatten()

rows = (rowsUpsc / upscaleMult).astype(np.int32)
cols = (colsUpsc / upscaleMult).astype(np.int32)

arrowOrigins = [[rowsUpsc[i], colsUpsc[i]] for i in range(len(rows))]
window_name="PyCFD"

cv2.namedWindow(window_name,cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, imageSize[0],imageSize[1])

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
video = cv2.VideoWriter('Output.mp4', fourcc,30, [imageSize[0], imageSize[1]], 0)
while True:
    #Simulation steps
    sim.timestep()
    dye[170:180,170:180] += 0.5
    dye = sim.advectField(dye, sim.u, sim.v)
    dye = smooth(dye)
    
    #Upscale
    scaledImage = cv2.resize(dye, imageSize)
    scaledImage = (scaledImage * 40).astype(np.uint8)
    u = (sim.u[rows, cols].flatten()*arrowSpacing*10).astype(np.int32)
    v = (sim.v[rows, cols].flatten()*arrowSpacing*10).astype(np.int32)
    endX = colsUpsc+u
    endY = rowsUpsc+v

    for i in range(len(rows)):
        cv2.arrowedLine(scaledImage, [colsUpsc[i], rowsUpsc[i]], [endX[i], endY[i]], 255)

    
    video.write(scaledImage)
    cv2.imshow(window_name, scaledImage)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video.release()
cv2.destroyAllWindows()